# HPC intro

## Jobs on an HPC system

When you log in via the terminal to an HPC system, you typically end up on a login node, i.e., a Linux system that gives you access to the HPC infrastructure.  Although you can run data processing scripts that take little time on a login node, this infrastructure is not intended for substantial computations.  Those are performed on compute nodes of the actual compute cluster.

To run an application on one or more compute nodes, you have two options:
  * a batch job,
  * an interactive job.

The vast majority of jobs are batch jobs, i.e., they run without user intervention.  These jobs are specified as a Bash script that contains some extra information on how you want to run your job.

This job script is submitted to a scheduler.  A scheduler is a software system that
  * knows the hardware characteristics (number of cores, memory, GPUs) of each node in the cluster;
  * the requirements of the jobs that have been submitted (requested number of nodes, cores, memory and run time)
  * the jobs that are currently running on the cluster.

Using this information, the cluster can efficiently schedule jobs on the cluster and make sure you cmoputations are executed.

In this tutorial, you will learn how to write a job script, how to submit it to the scheduler, how to monitor and manage your jobs.

## Job scripts

A job script describes the work you want to do, and the resources that are required to do that.  Here you will learn about the typical anatomy of such a script.

### Shebang

  A job script is a Bash script, so like any such script, its first line is a shebang.

```bash
#!/usr/bin/env bash
```

This line tells the scheduler that this is a job script that should be executing using Bash.

### Scheduler directives

In order for the scheduler to handle your job correctly, you will have to provide some information as scheduler directives.

#### Credit account

If your HPC center uses a credit system, you have to specify a credit account you can access.  Say the name of that account is `lp_tutorial`, you would specify that as follows.

```bash
#SBATCH --account=lp_tutorial
```

You can check which credit accounts you have access to using the `sam-balance` command.

In [ ]:
sam-balance

If you don't see any credit accounts, you will have to apply for credits before proceding with this tutorial.  You can check how to do that in the documentation if you have a
  * [KU Leuven affiliation](https://docs.vscentrum.be/en/latest/leuven/credits.html?highlight=credits#ku-leuven-users), or
  * [Hasselt University affiliation](https://docs.vscentrum.be/en/latest/leuven/credits.html?highlight=credits%20uhasselt#uhasselt-users).

#### Nodes and tasks

Next, you specify the resources you need using scheduler directives, i.e., lines that start with `#SBATCH`.  For example, you want your job to run on a single node, there is only a single task to do, and that task requires a single core.

```bash
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
```

#### Memory requirements

You may also want to specify the memory usage of your application. Again, do not underestimate, but also keep in mind the hardware characteristics of the compute nodes.  If you request more memory than a node has, your job will not run.  For instance, if you are sure your job can run with 3 GB of RAM, you can specify that as follows.

```bash
#SBATCH --mem=3g
```

#### Walltime

You would also have to specify the maximum time your script should take to run.  You have to realize that the execution of your script will be terminated once that time has elapsed, so do not underestimate it.  Time is specified as "HHH:MM:SS", for example, if your computation would take at most 2 hours, you would specify that as follows.

```bash
#SBATCH --time=02:00:00
```

### Commands

Up to this point, you have only specified what the scheduler should know to handle your job, but not yet what is supposed bo be computed.  Since this is a very general introduction to the job system, you will first experiment with something really simple, and after that, you can move on to domain-specific computations.

You want to compute the product of pair of integers in the range 1 to 10.  In Bash, that can be done using the following code.

```bash
# actual computation, a bit boring
for i in $(seq 1 10)
do
    for j in $(seq 1 10)
    do
        echo $(( $i * $j ))
    done
done
```

Clearly, this is not exactly an HPC-caliber computation, but again, this is just a generic example.e

### Putting it all together

You can put all of this together in a single file `jobscript.slurm`.

```bash
#!/usr/bin/env bash
#SBATCH --accoun=lp_tutorial
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:05:00

# don't put sleep in your job scripts, it is only added here to ensure
# that the job will run for about 2 minutes, which gives you time to
# monitor it with squeue after submission, see below.
sleep 120

# actual computation, a bit boring
for i in $(seq 1 10)
do
    for j in $(seq 1 10)
    do
        echo $(( $i * $j ))
    done
done
```

Note that
  * you have to replace `lp_tutorial` by an account you have access to;
  * the memory has been adjusted to 1 GB;
  * the walltime has been adjusted to 2 minutes.

## Job life cycle

The life cycle of a job consists of a number of steps that you will learn about below.

### Job submission

Your job script is now ready to be submitted to the scheduler.  You can do that using `sbatch`.  You should specify the cluster you want the job to run on, `wice` in this case.

In [ ]:
sbatch  --cluster=wice  jobscript.slurm

If all goes well, `sbatch` will write the job ID to standard output.  Job IDs are unique, and you can use them to monitor the status of your job, to cancel a job if necessary, or to retrieve information about it while or after it finishes.

In this case, all the information required by slurm, the scheduler, is present in the job script as slurm directives (`#SBATCH`).  However, if that is not the case, or if you want different values, you can add those as command line options when you invoke `sbatch`.  For instance, to give your job a name, say "multiplication table", you can use the `--job-name` option.

In [ ]:
sbatch  --cluster=wice  --job-name='multiplication table'  jobscript.slurm

### Job status

Of course, you would like to keep an eye on your job(s).  You want to know whether they are still waiting to start, are running, or are completed.  The command to get this information is `squeue`.  Note that you have to specify the cluster you want to check.

In [ ]:
squeue  --cluster=wice

You will get a list of jobs that are
  * pending: these jobs are not yet running, the scheduler will start them when resources are available;
  * running: these jobs are executing;
  * copmleting: these jobs are about to finish.

When you have submitted a job, and you don't see it anymore, it completed.

The output of `squeue` contains a lot of information:
  * `JOBID`: this is the unique ID assigned to your job;
  * `PARTITION`: a system may have multiple partitions that you can use depending on the job characterisitcs,
    jobs you submit using `sbatch` typically end up in the `batch` partition;
  * `NAME`: the name you gave to your job, or the one assigned to it by slurm if you didn't specify any;
  * `USER`: your user ID;
  * `ST`: the state of the job, the once you see most often are
    * `PD`: pending, i.e., waiting to be started by the scheduler;
    * `R`: running, i.e., your job is being executed;
    * `CG`: completing, i.e., your job is about to finish;
  * `TIME`: the time the job has been running, so `0:00` for pending jobs;
  * `NODES`: the number of nodes the job requires;
  * `NODELIST(REASON)`: for running jobs, the nanme(s) of the node(s) the job is running on,
    for pending jobs, the reason why the job is not started yet.

### Job completion and output

When you no longer see your job listed in `squeue`'s output, your job has completed.  You will find a new file in the directory you submitted your job in.  The name of the file starts with `slurm-`, followed by the job ID and `.out` as extension.

This file contains everyting your job script has writting to standard output and standard error.  It is a text file, so you can inspect it using your favorite editor or using a pager such as `less`.

Below you see a fragment of the output file for a job like the one you just submitted.

```
SLURM_JOB_ID: 60559573
SLURM_JOB_USER: vsc30032
SLURM_JOB_ACCOUNT: lpt2_sysadmin
SLURM_JOB_NAME: jobscript.slurm
SLURM_CLUSTER_NAME: wice
SLURM_JOB_PARTITION: batch
SLURM_NNODES: 1
SLURM_NODELIST: l33c32n2
SLURM_JOB_CPUS_PER_NODE: 1
Date: Thu Jul 27 09:44:21 CEST 2023
Walltime: 00-00:02:00
========================================================================
1
2
3
4
5
6
7
8
9
10
2
4
...
```

The file consists of two sections.  The first contains information about your job.  The second section, below the line of '=' characters is the actual standard output and standard error of your job, in this case our multiplication table.

There are a few options to control this output:
  * `--output=file-name`: standard output will be saved in the file with a name you provide;
  * `--error=file-name`: standard error will be saved in the file with the name you providde.

In fact, the string(s) you specify with `--output` and/or `--error` is interpreted as a file name pattern.  You can use, e.g.,
```bash
#SBATCH --output=multiplication_table_%j.out
#SBATCH --error=multiplication_table_%j.err
```
In this pattern `%j` will be replaced by the job ID.

It is considered good practice to include the job ID in the output/error file names since this makes it a lot easier to diagnose problems with a job, and link the output to other job-related information.

### Information on jobs

You may want to get some useful information on jobs, such as the walltime they used to complete, the total CPU time they consumed, or the memory they required.  This information is available through the `sacct` command (slurm accounting).  Replace the job ID in the command below by the one of the job you just ran.

In [ ]:
sacct --jobs 60559575

As you can see, by default the only useful inforamtion you get is
  * `Account`: the credit account used for the job;
  * ``AllocCPUS`: the number of CPUs (i.e., cores) allocated for your job;
  * `State`: the state of your job; and
  * `ExitCode`: this will be 0 if your job completed succesfully, non-zero otherwise.
  
Note that an exit code 0 doesn't necessarily mean that everything went well.  It is simply the exit code of the last Bash statement executed in your job script.  **Always check your job's output file and error file, if any.***

To get more details on your job, you can specify that you want particular metrics to be displayed when running `sacct`.  For instance, if you want to see the walltime (`Elapsed`), total CPU time (`CPUTime`), memory used (`MaxRSS`) your would use the `--format` option.

In [ ]:
sacct  --jobs=60559575  --format=JobID,Elapsed,CPUTime,MaxRSS

To get a list of all the information that `sacct` can provide on a job, use the `--helpformat` option.

In [ ]:
sacct  --helpformat

**Tip:** since it is not really convenient to remember and type the `--format` option each time you use `sacct`, you can define the environment variable `SACCT_FORMAT`, e.g.,

In [ ]:
export SACCT_FORMAT='JobID,Elapsed,CPUTime,MaxRSS,State'

In [ ]:
sacct --jobs 60559573

If you no longer want to use the format defined in `SACCT_FORMAT`, you can unset the variable.

In [ ]:
unset SACCT_FORMAT 

### Cancelling jobs

You may want to cancel a job, either while it is still pending, or when it is already running.  For instance, you may realize that you made a mistake, and that the job would not produce the results you want.  In that case, you can use the `scancel` command.

First, submit a job, so that there is something to cancel.

In [ ]:
sbatch  --cluster=wice  jobscript.slurm

Now you can cancel it, but don't forget to replace the job ID!

In [ ]:
scancel 60559656 

Verify the job was cancelled.

In [ ]:
sacct --jobs 60559656

Obviously, you can not cancel a job that already completed.

## More examples

The job script you used in this tutorial is of course a very simple one that only executes some (pretty boring) Bash commands.  In practice, you would like for instance to run a Python or an R script, so you will find job scripts to run the scripts you wrote in those tutorials as jobs on the cluster.

### Running Python jobs

In the [tutorial on running Python scripts](005_running_python.ipynb) you created a [script to create a plot of a sine function](005_artefacts/sin_plot.py).

If you don't have the script in this directory, you can copy it from the `005_artefacts` directory.

The job script to run this Python script is given below.

```bash
#!/usr/bin/env bash

#SBATCH --account=lp_tutorial
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:02:00

# set up the environment by loading the required modules
module purge
module load SciPy-bundle/2021.05-foss-2021a
module load matplotlib/3.4.2-foss-2021a

# run the Python script
python sin_plot.py
```

Again, remember to replace the account by one you have access to.

Note that you need to load all the required modules in your job script, in this case the Scipy-bundle and matplotlib modules.

### Running R jobs

In the [tutorial on running R scripts](007_running_R.ipynb) you created a [script to create a plot of the Airy function](07_artefacts/airy_plot.R).

If you don't have the script in this directory, you can copy it from the `007_artefacts` directory.

The job script to run this R script is given below.

```bash
#!/usr/bin/env bash

#SBATCH --account=lp_tutorial
#SBATCH --nodes=1
#SBATCH --ntasks=1
#SBATCH --cpus-per-task=1
#SBATCH --mem=1g
#SBATCH --time=00:02:00

# set up the environment by loading the required modules
module purge
module load R/4.1.0-foss-2021a
module load GSL/2.7-GCC-10.3.0

# run the R script
Rscript airy_plot.R
```

Again, remember to replace the account by one you have access to.

Note that in addition to the R module, you also have to load the GSL module since the R library you use in this script requires it at runtime.

## Summary

In this tutorial, you learned about the basics of job scripts and the scheduler, now you know how to use the cluster in its normal mode of operation, i.e., batch mode, submitting jobs to the scheduler.

You learned
  * how to write a job script by
    * specifying directives for the scheduler,
    * specifying the commands you want to be executed in your job;
  * to submit a job to the scheduler using `sbatch`;
  * to monitor the state of your job using `squeue`;
  * to get information on your job using `sacct`;
  * to cancel a job using `scancel`.

## Where to go from here?

There is of course much more to learn about running jobs on an HPC system.